In [5]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

warnings.filterwarnings('ignore')

### Load Data

In [6]:
causeway_stockpoint = pd.read_feather('./output/causeway_stockpoint.feather')
depot_location = [causeway_stockpoint.Latitude[0], causeway_stockpoint.Longitude[0]]

In [7]:
causeway = pd.read_excel('./output/causeway.xlsx')

df_customer_dim_with_affinity_score = pd.read_feather('./input/customer_dim_with_affinity_score.feather').rename(columns={'FCID':'Stock_Point_ID'})
df_customer_dim_with_affinity_score['Stock_Point_ID'] = df_customer_dim_with_affinity_score['Stock_Point_ID'].astype(int)

col1 = ['Latitude', 'Longitude','TotalSKUs', 'AvgSKUScore', 'TotalEstimatedVolume', 'RFcount', 'HighValueSKUs', 
        'HighValueAvgScore', 'HighValueTotalScore', 'HighValueEstimatedVolume', 'ExpressSKUs', 
        'CoreSKUs', 'CustomerAffinityScore_Raw', 'CustomerAffinityScore_Standardized', 'CustomerAffinityRank']

for col in col1: 
    df_customer_dim_with_affinity_score[col] = pd.to_numeric(df_customer_dim_with_affinity_score[col], errors='coerce').fillna(0)

   
# Replace invalid latitude values with NaN
df_customer_dim_with_affinity_score.loc[
    (df_customer_dim_with_affinity_score['Latitude'] < -90) |
    (df_customer_dim_with_affinity_score['Latitude'] > 90),
    'Latitude'
] = 0.0

df_customer_dim_with_affinity_score.loc[
    (df_customer_dim_with_affinity_score['Longitude'] < -180) |
    (df_customer_dim_with_affinity_score['Longitude'] > 180),
    'Longitude'
] = 0.0   

customers_df = df_customer_dim_with_affinity_score.merge(causeway[['CustomerID']].drop_duplicates(), how='inner', on='CustomerID')

### Main

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore')

# Generate sample customer data
def generate_sample_data(n_customers=100, city_center=(40.7589, -73.9851), spread_km=20):
    """Generate sample customer data around a city center."""
    np.random.seed(42)
    
    # Generate random coordinates around city center
    # Convert km to approximate degrees (rough approximation)
    lat_spread = spread_km / 111.0  # 1 degree lat ≈ 111 km
    lon_spread = spread_km / (111.0 * np.cos(np.radians(city_center[0])))
    
    latitudes = np.random.normal(city_center[0], lat_spread/3, n_customers)
    longitudes = np.random.normal(city_center[1], lon_spread/3, n_customers)
    
    # Create customer IDs
    customer_ids = [f"CUST_{i:03d}" for i in range(1, n_customers + 1)]
    
    return pd.DataFrame({
        'CustomerID': customer_ids,
        'Latitude': latitudes,
        'Longitude': longitudes
    })

In [ ]:
# Create sample data
sample_customers = generate_sample_data(n_customers=150)
print("Sample data created:")
print(sample_customers.head())
print(f"Total customers: {len(sample_customers)}")

## 1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER

In [ ]:
# =============================================================================
# 1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER
# =============================================================================

from clustering.geographic_clustering import GeographicClusteringOptimizer  # Your original code

print("\n" + "="*60)
print("1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER")
print("="*60)

# Initialize with default parameters
optimizer = GeographicClusteringOptimizer()

print("\nDEFAULT PARAMETERS:")
print("- max_customers_per_cluster=20")
print("- max_distance_km=50")
print("- dbscan_eps_km=10")
print("- dbscan_min_samples=3")
print("- use_haversine=True")

# Example 1: Hierarchical clustering
print("\n1a. Hierarchical Clustering (Distance-based):")
hierarchical_result = optimizer.create_hierarchical_clusters(
    sample_customers.copy(),
    method='average',      # OPTIONAL: 'average', 'ward', 'single', 'complete'
    criterion='distance'   # OPTIONAL: 'distance', 'maxclust', 'inconsistent'
)
print(f"Clusters created: {hierarchical_result['cluster'].nunique()}")
print(f"Cluster sizes: {hierarchical_result['cluster'].value_counts().sort_index().head()}")

# Example 2: DBSCAN clustering
print("\n1b. DBSCAN Clustering:")
dbscan_result = optimizer.create_dbscan_clusters(
    sample_customers.copy(),
    return_noise_separately=False  # OPTIONAL: True/False
)
print(f"Clusters created: {dbscan_result['cluster'].nunique()}")

# Example 3: Hybrid clustering
print("\n1c. Hybrid Clustering:")
hybrid_result = optimizer.create_hybrid_clusters(
    sample_customers.copy(),
    dbscan_first=True,                    # OPTIONAL: True/False
    hierarchical_method='average',        # OPTIONAL: linkage method
    hierarchical_criterion='distance'     # OPTIONAL: clustering criterion
)
print(f"Clusters created: {hybrid_result['cluster'].nunique()}")

# Custom parameters example
print("\n1d. Custom Parameters Example:")
custom_optimizer = GeographicClusteringOptimizer(
    max_customers_per_cluster=15,  # REQUIRED
    max_distance_km=30,           # REQUIRED  
    dbscan_eps_km=8,              # OPTIONAL (default=10)
    dbscan_min_samples=2,         # OPTIONAL (default=3)
    use_haversine=False           # OPTIONAL (default=True)
)

custom_result = custom_optimizer.create_hierarchical_clusters(sample_customers.copy())
print(f"Custom clusters created: {custom_result['cluster'].nunique()}")


## 2. DIVISIVE HIERARCHICAL CLUSTERING

In [18]:
# =============================================================================
# 2. DIVISIVE HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.divisive_clustering import DivisiveGeographicClustering, OptimizedDivisiveGeographicClustering

print("\n" + "="*60)
print("2. DIVISIVE HIERARCHICAL CLUSTERING")
print("="*60)

divisive_clusterer = OptimizedDivisiveGeographicClustering(
    max_customers_per_cluster=20,  # REQUIRED
    max_distance_km=30            # REQUIRED
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- max_distance_km=50 (REQUIRED)")

divisive_result = divisive_clusterer.divisive_clustering(customers_df.copy())
print(f"\nDivisive clusters created: {divisive_result['cluster'].nunique()}")
print(f"Cluster sizes: {divisive_result['cluster'].value_counts().sort_index().head()}")


2. DIVISIVE HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- max_distance_km=50 (REQUIRED)

Divisive clusters created: 106
Cluster sizes: cluster
1    3
2    1
3    2
4    1
5    8
Name: count, dtype: int64


In [19]:
divisive_result.to_feather('./results/divisive_result.feather')

In [18]:
divisive_result.cluster.value_counts().reset_index(name='ncustomers')

,cluster,ncustomers
0,98,20
1,26,20
2,38,20
3,63,20
4,105,20
...,...,...
101,2,1
102,74,1
103,9,1
104,8,1


In [19]:
from clustering.plot_cluster import create_cluster_map

map_clusters = create_cluster_map(
    divisive_result,
    popup_cols=['CustomerID', 'LGA', 'LCDA'],
    tooltip_cols=['LGA', 'LCDA']
)
map_clusters

## 3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING

In [ ]:
%load_ext autoreload
%autoreload 2

from clustering.constrained_hierarchical_clustering import ConstraintBasedClustering , OptimizedConstraintBasedClustering

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# =============================================================================
# 3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING
# ============================================================================= 

from clustering.constrained_hierarchical_clustering import ConstraintBasedClustering, OptimizedConstraintBasedClustering

print("\n" + "="*60)
print("3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING")
print("="*60)

constraint_clusterer = OptimizedConstraintBasedClustering(
    max_customers_per_cluster=20,  # REQUIRED
    max_distance_km=50,           # REQUIRED
    max_route_distance_km=100     # OPTIONAL (default=100)
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- max_distance_km=50 (REQUIRED)")
print("- max_route_distance_km=100 (OPTIONAL)")

constraint_result = constraint_clusterer.constrained_hierarchical_clustering_optimized(customers_df.copy())
print(f"\nConstraint-based clusters created: {constraint_result['cluster'].nunique()}")
print(f"Cluster sizes: {constraint_result['cluster'].value_counts().sort_index().head()}")


3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- max_distance_km=50 (REQUIRED)
- max_route_distance_km=100 (OPTIONAL)

Constraint-based clusters created: 62
Cluster sizes: cluster
1    1
2    1
3    1
4    1
5    1
Name: count, dtype: int64


In [24]:
constraint_result.cluster.value_counts().reset_index(name='ncustomer')

constraint_result.to_feather('./results/constraint_result.feather')

## 4. DENSITY-BASED HIERARCHICAL CLUSTERING

In [41]:
# =============================================================================
# 4. DENSITY-BASED HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.density_hierarchical_clustering import DensityHierarchicalClustering

print("\n" + "="*60)
print("4. DENSITY-BASED HIERARCHICAL CLUSTERING")
print("="*60)

density_clusterer = DensityHierarchicalClustering(
    max_customers_per_cluster=20,  # REQUIRED
    min_cluster_size=3,           # OPTIONAL (default=3)
    k_neighbors=10                 # OPTIONAL (default=5)
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- min_cluster_size=3 (OPTIONAL)")
print("- k_neighbors=5 (OPTIONAL)")

density_result = density_clusterer.build_density_hierarchy(customers_df.copy())
print(f"\nDensity-based clusters created: {density_result['cluster'].nunique()}")
print(f"Cluster sizes: {density_result['cluster'].value_counts().sort_index().head()}")



4. DENSITY-BASED HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- min_cluster_size=3 (OPTIONAL)
- k_neighbors=5 (OPTIONAL)

Density-based clusters created: 82
Cluster sizes: cluster
-1    80
 1    10
 2    10
 3    10
 4    10
Name: count, dtype: int64


In [42]:
density_result.to_feather('./results/density_result.feather')
density_result.cluster.value_counts().reset_index(name='ncustomer').query('ncustomer > 5').head()

,cluster,ncustomer
0,-1,80
1,40,20
2,38,20
3,30,20
4,42,20


In [43]:
from clustering.density_hierarchical_clustering import OptimizedDensityHierarchicalClustering

adaptive_density_clusterer =  OptimizedDensityHierarchicalClustering(
                max_customers_per_cluster=20, 
                min_cluster_size=3, 
                 k_neighbors=10, 
                 density_threshold=0.5)


print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- min_cluster_size=3 (OPTIONAL)")
print("- k_neighbors=5 (OPTIONAL)")

adaptive_density_result = adaptive_density_clusterer.build_density_hierarchy_adaptive(customers_df.copy())
print(f"\nDensity-based clusters created: {density_result['cluster'].nunique()}")
print(f"Cluster sizes: {density_result['cluster'].value_counts().sort_index().head()}")



PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- min_cluster_size=3 (OPTIONAL)
- k_neighbors=5 (OPTIONAL)

Density-based clusters created: 82
Cluster sizes: cluster
-1    80
 1    10
 2    10
 3    10
 4    10
Name: count, dtype: int64


In [44]:
adaptive_density_result.to_feather('./results/adaptive_density_result.feather')
adaptive_density_result.cluster.value_counts().reset_index(name='ncustomer').query('ncustomer > 5').head()

,cluster,ncustomer
0,35,20
1,42,20
2,23,20
3,38,20
4,31,20


## 5. ROUTE-AWARE CLUSTERING

In [52]:
# =============================================================================
# 5. ROUTE-AWARE CLUSTERING
# =============================================================================
from clustering.route_aware_clustering import RouteAwareClustering, OptimizedRouteAwareClustering
print("\n" + "="*60)
print("5. ROUTE-AWARE CLUSTERING")
print("="*60)
 

route_clusterer = OptimizedRouteAwareClustering(
    max_customers_per_route=20,      # REQUIRED
    max_route_time_minutes=480,      # OPTIONAL (default=480, 8 hours)
    avg_service_time_minutes=15,     # OPTIONAL (default=15)
    avg_speed_kmh=30,               # OPTIONAL (default=30)
    depot_location=depot_location    # OPTIONAL (default=None)
)

print("\nPARAMETERS:")
print("- max_customers_per_route=20 (REQUIRED)")
print("- max_route_time_minutes=480 (OPTIONAL)")
print("- avg_service_time_minutes=15 (OPTIONAL)")
print("- avg_speed_kmh=30 (OPTIONAL)")
print("- depot_location=(lat, lon) (OPTIONAL)")

# Example 5a: Sweep Algorithm
print("\n5a. Sweep Algorithm Clustering:")
sweep_result = route_clusterer.sweep_algorithm_clustering(customers_df.copy())
# sweep_result['cluster'] = sweep_result['cluster'] + 1
print(f"Sweep clusters created: {sweep_result['cluster'].nunique()}")
print(f"Cluster sizes: {sweep_result['cluster'].value_counts().sort_index().head()}")



5. ROUTE-AWARE CLUSTERING

PARAMETERS:
- max_customers_per_route=20 (REQUIRED)
- max_route_time_minutes=480 (OPTIONAL)
- avg_service_time_minutes=15 (OPTIONAL)
- avg_speed_kmh=30 (OPTIONAL)
- depot_location=(lat, lon) (OPTIONAL)

5a. Sweep Algorithm Clustering:
Sweep clusters created: 105
Cluster sizes: cluster
1    20
2    20
3    20
4    20
5    20
Name: count, dtype: int64


In [53]:
sweep_result.to_feather('./results/sweep_result.feather')
sweep_result.cluster.value_counts().reset_index(name='ncustomer').query('ncustomer > 5').head()

,cluster,ncustomer
0,89,20
1,60,20
2,10,20
3,58,20
4,80,20


In [55]:
# Example 5b: Capacity Constrained Clustering
print("\n5b. Capacity Constrained Clustering:")
capacity_result = route_clusterer.capacity_constrained_clustering(customers_df.copy())
# sweep_result['cluster'] = sweep_result['cluster'] + 1
print(f"Capacity-constrained clusters created: {capacity_result['cluster'].nunique()}")
print(f"Cluster sizes: {capacity_result['cluster'].value_counts().sort_index().head()}")


5b. Capacity Constrained Clustering:
Capacity-constrained clusters created: 101
Cluster sizes: cluster
1    20
2    20
3    20
4    20
5    20
Name: count, dtype: int64


In [57]:
capacity_result.to_feather('./results/capacity_result.feather')
capacity_result.cluster.value_counts().reset_index(name='ncustomer').query('ncustomer > 5').head()

,cluster,ncustomer
0,13,20
1,10,20
2,25,20
3,44,20
4,40,20


In [65]:
# Example without depot location
print("\n5c. Route-Aware Clustering without depot:")
no_depot_clusterer = OptimizedRouteAwareClustering(
    max_customers_per_route=20,
    max_route_time_minutes=360,
    depot_location=None  # Will use geographic center
)
no_depot_result = no_depot_clusterer.capacity_constrained_clustering(customers_df.copy())
print(f"No-depot clusters created: {no_depot_result['cluster'].nunique()}")
print(f"Cluster sizes: {no_depot_result['cluster'].value_counts().sort_index().head()}")


5c. Route-Aware Clustering without depot:
No-depot clusters created: 59
Cluster sizes: cluster
1    20
2    20
3    20
4    20
5    20
Name: count, dtype: int64


In [66]:
no_depot_result.to_feather('./results/no_depot_result.feather')
no_depot_result.cluster.value_counts().reset_index(name='ncustomer').query('ncustomer > 5').head()

,cluster,ncustomer
0,1,20
1,2,20
2,3,20
3,4,20
4,5,20


## COMPARISON SUMMARY

In [68]:
# =============================================================================
# COMPARISON SUMMARY
# =============================================================================

print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

results_summary = {
    'Method': [
        # 'Hierarchical (Distance)',
        # 'DBSCAN',
        # 'Hybrid',
        'Divisive',
        'Constraint-Based',
        'Density-Based',
        'Adaptive-Density-Based',
        'Sweep Algorithm',
        'Capacity Constrained',
        'No-Depo Constrained',
    ],
    'Clusters': [
        # hierarchical_result['cluster'].nunique(),
        # dbscan_result['cluster'].nunique(),
        # hybrid_result['cluster'].nunique(),
        divisive_result['cluster'].nunique(),
        constraint_result['cluster'].nunique(),
        density_result['cluster'].nunique(),
        adaptive_density_result['cluster'].nunique(),
        sweep_result['cluster'].nunique(),
        capacity_result['cluster'].nunique(),
        no_depot_result['cluster'].nunique()
    ]
}

summary_df = pd.DataFrame(results_summary)
print(summary_df)




COMPARISON SUMMARY
                   Method  Clusters
0                Divisive       106
1        Constraint-Based        62
2           Density-Based        82
3  Adaptive-Density-Based       136
4         Sweep Algorithm       105
5    Capacity Constrained       101
6     No-Depo Constrained        59


In [ ]:
from sklearn.metrics import silhouette_score
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

def evaluate_clustering(df,
                        id_col = 'CustomerID'
                        ,cluster_col = 'cluster'
                        ,lat_col = 'Latitude'
                        ,lon_col = 'Longitude'
                        ):
    # Check input
    required_cols = {id_col, cluster_col, lat_col, lon_col}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"Input DataFrame must contain columns: {required_cols}")

    # Cluster size distribution
    cluster_counts = df['cluster'].value_counts().sort_index()

    # Silhouette score
    coords = df[['Latitude', 'Longitude']].values
    labels = df['cluster'].values
    unique_clusters = np.unique(labels)
    
    if len(unique_clusters) > 1 and len(unique_clusters) < len(df):
        silhouette = silhouette_score(coords, labels)
    else:
        silhouette = np.nan  # Not enough clusters for silhouette

    # Compute cluster centroids
    centroids = df.groupby('cluster')[['Latitude', 'Longitude']].mean()

    # Intra-cluster distances
    intra_cluster_distances = {}
    for cluster_id in unique_clusters:
        cluster_points = df[df['cluster'] == cluster_id][['Latitude', 'Longitude']].values
        centroid = centroids.loc[cluster_id].values
        distances = cdist(cluster_points, [centroid])
        intra_cluster_distances[cluster_id] = distances.mean()

    # Return summary
    return {
        "cluster_counts": cluster_counts,
        "silhouette_score": silhouette,
        "centroids": centroids,
        "intra_cluster_distances": intra_cluster_distances
    }


from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

def evaluate_unsupervised_clustering(df):
    # Usage:
    X = df[['Latitude', 'Longitude']].values
    labels = df['cluster'].values
    scores = {
        "Silhouette Score": silhouette_score(X, labels).round(2),
        "Davies-Bouldin Index": davies_bouldin_score(X, labels).round(2),
        "Calinski-Harabasz Score": calinski_harabasz_score(X, labels).round(2)
    }
    return scores
   
all_result = {}
for name, df in {'divisive_result':divisive_result, 'constraint_result':constraint_result, 'density_result':density_result,
                 'adaptive_density_result':adaptive_density_result, 'sweep_result':sweep_result, 'capacity_result':capacity_result, 
                 'no_depot_result':no_depot_result }.items():
    c_summary = df.cluster.value_counts().reset_index(name='size').query('size > 5')
    df_ = df[df.cluster.isin(c_summary.cluster)]
    # result = evaluate_clustering(df_)
    result = evaluate_unsupervised_clustering(df_)
    print(f"""{name} : Silhouette Score: {result}""")

## PARAMETER REFERENCE GUIDE

In [ ]:
# =============================================================================
# PARAMETER REFERENCE GUIDE
# =============================================================================

print("\n" + "="*60)
print("PARAMETER REFERENCE GUIDE")
print("="*60)

parameter_guide = """
1. GEOGRAPHIC CLUSTERING OPTIMIZER:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: dbscan_eps_km=10, dbscan_min_samples=3, use_haversine=True
   
2. DIVISIVE CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: None
   
3. CONSTRAINT-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: max_route_distance_km=100
   
4. DENSITY-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster
   OPTIONAL: min_cluster_size=3, k_neighbors=5
   
5. ROUTE-AWARE CLUSTERING:
   REQUIRED: max_customers_per_route
   OPTIONAL: max_route_time_minutes=480, avg_service_time_minutes=15,
            avg_speed_kmh=30, depot_location=None

METHOD-SPECIFIC OPTIONAL PARAMETERS:
- create_hierarchical_clusters(): method='average', criterion='distance'
- create_dbscan_clusters(): return_noise_separately=False
- create_hybrid_clusters(): dbscan_first=True, hierarchical_method='average',
                           hierarchical_criterion='distance'
"""

print(parameter_guide)



## PRACTICAL USAGE EXAMPLES

In [ ]:
# =============================================================================
# PRACTICAL USAGE EXAMPLES
# =============================================================================

print("\n" + "="*60)
print("PRACTICAL USAGE EXAMPLES")
print("="*60)

print("\nExample 1: Quick start with minimal parameters")
print("=" * 50)
quick_optimizer = GeographicClusteringOptimizer(
    max_customers_per_cluster=15,
    max_distance_km=25
)
quick_result = quick_optimizer.create_hierarchical_clusters(sample_customers.copy())
print(f"Quick clusters: {quick_result['cluster'].nunique()}")

print("\nExample 2: Delivery-focused clustering")
print("=" * 50)
delivery_optimizer = RouteAwareClustering(
    max_customers_per_route=12,
    max_route_time_minutes=300,  # 5 hours
    depot_location=(40.7589, -73.9851)
)
delivery_result = delivery_optimizer.capacity_constrained_clustering(sample_customers.copy())
print(f"Delivery-optimized clusters: {delivery_result['cluster'].nunique()}")

print("\nExample 3: Dense urban area clustering")
print("=" * 50)
urban_optimizer = DensityHierarchicalClustering(
    max_customers_per_cluster=25,
    min_cluster_size=5,
    k_neighbors=8
)
urban_result = urban_optimizer.build_density_hierarchy(sample_customers.copy())
print(f"Urban density clusters: {urban_result['cluster'].nunique()}")

print("\n" + "="*60)
print("EXECUTION COMPLETED SUCCESSFULLY!")
print("="*60)